In [1]:
#lstm-10days
#인풋데이터가 잘못되어 revision 


import pandas as pd
import numpy as np
'''
data preprocessing
'''

#주가데이터 파일을 로드
csv_data = pd.read_csv("./data/005930_200325.csv")
csv_data

,date,start,high,low,end,volume,foreign,person,company,wondollar
0,2020.3.25,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,2020.3.24,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,2020.3.23,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,2020.3.20,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,2020.3.19,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...
592,2017.10.23,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,2017.10.20,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,2017.10.19,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,2017.10.18,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [2]:
#필요한 부분을 가져온다
stdata = csv_data[["start","high", "low","end","volume","foreign", "person","company","wondollar"]]
stdata.head()

#전일대비 가격 변화율을 추가한다
price_change = -np.diff(stdata["end"].to_numpy())
price = stdata["end"].to_numpy()
price = price[1:]
price_change_rate = (price_change/price)*100
price_change_rate=np.append(price_change_rate,[0])
stdata['price_change_rate'] = price_change_rate

#캔들 (종가,시가,고가,저가) 관련 정보를 추가한다
#시가-종가
start = stdata["start"].to_numpy()
end = stdata["end"].to_numpy()
candle_end_start = end-start
stdata['candle_end_start'] = candle_end_start

#고가-저가 (당일변동폭) 
high = stdata["high"].to_numpy()
low = stdata["low"].to_numpy()
candle_high_low = high-low
stdata['candle_high_low'] = candle_high_low


#상승or하락 표시 상승:1 보함:0 하락:-1
up_or_down = np.zeros(597)

price_change_rate = stdata["price_change_rate"].to_numpy()
for idx, val in enumerate(price_change_rate):
    if val>0:
        up_or_down[idx]=1
    if val<0:
        up_or_down[idx]=-1
stdata['up_or_down'] = up_or_down

stdata


,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [3]:
#사용할 feature를 선택한다

input_data = stdata[["price_change_rate","candle_end_start","candle_high_low","up_or_down","start","high", "low","end","volume","foreign", "person","company","wondollar"]]
input_data

,price_change_rate,candle_end_start,candle_high_low,up_or_down,start,high,low,end,volume,foreign,person,company,wondollar
0,3.620873,-300,2450,1.0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,10.470588,3100,3900,1.0,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,-6.387665,-100,1150,-1.0,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,5.704307,1250,1950,1.0,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,-5.811404,-3450,4350,-1.0,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,0.854383,-300,640,1.0,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,1.623254,1040,1300,1.0,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,-3.250548,-1720,1720,-1.0,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,-0.072993,-60,1200,-1.0,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [4]:
#정규화 안할경우
input_arr = input_data.to_numpy()
input_arr.shape

(597, 13)

In [5]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fitted = scaler.fit(input_data)
input_arr = scaler.transform(input_data)
input_arr.shape

(597, 13)

In [6]:
#역순
input_arr = np.flip(input_arr, axis=0)
print(input_arr.shape)
#input_arr

(597, 13)


In [7]:
#정답추출
up_or_down_ans = input_arr[:,3]

up_or_down_ans = up_or_down_ans[1:]
print(up_or_down_ans.shape)
#up_or_down_ans

(596,)


In [8]:
#마지막 행은 정답이 없어 버린다
input_arr = input_arr[:-1]
print(input_arr.shape)
#input_arr

(596, 13)


In [9]:
#1일 -> len_of_day일 단위의 데이터로 변환 
len_of_day =10
feature_count = input_arr.shape[1]




#596일의 데이터로 10일짜리 데이터를 생성하면 596-10+1=587 개의 데이터가 생성된다
input_data_count = input_arr.shape[0]-len_of_day+1



#596*6열     [[feature갯수 개], 
#             [feature갯수개], 
#             [feature갯수개],
#             [feature갯수개],
#              ... ]
#
# 
#              을 [    [ [feature갯수],[feature갯수],[feature갯수]..........[feature갯수] (feature갯수개짜리 n일) ],
#                [ (feature갯수짜리 n일개) ],
#                [ (13개짜리 n일개) ],
#               .
#               .
#               . 
#               582개 
#    
#            ]  ( 582  ,10(일수) , 6(feature갯수) )    형태로 만들어야 한다


#inputdata1 : 10일 짜리 데이터 생성 

inputdata1 = np.empty((input_data_count,len_of_day,feature_count), dtype=np.float64)

for i in range(input_data_count):
    for j in range(len_of_day):
        inputdata1[i][j]=input_arr[i+j]
        
inputdata1.shape 

(587, 10, 13)

In [10]:
dummy_input = np.array([ [ [1,1], [0,0], [1,1] ],
                         [ [0,0], [1,1], [1,0] ], 
                         [ [0,0], [1,1], [0,1] ],
                         [ [1,0], [0,0], [1,1] ],
                       ], dtype =np.float32)

dummy_input.shape

(4, 3, 2)

In [11]:
up_or_down_ans_dummy = np.array([1,0,0,1])
up_or_down_ans_dummy = up_or_down_ans_dummy.reshape(-1,1)
up_or_down_ans_dummy.shape

(4, 1)

In [12]:
up_or_down_ans = up_or_down_ans[len_of_day - 1:]
print(up_or_down_ans.shape)
up_or_down_ans=up_or_down_ans.reshape(-1,1)
print(up_or_down_ans.shape)
#up_or_down_ans

(587,)
(587, 1)


In [13]:
#up_or_down_ans

In [14]:
for idx, data in enumerate(up_or_down_ans) :
    if data <0.6:
        up_or_down_ans[idx] = 0
#up_or_down_ans       

In [15]:
up_or_down_ans = up_or_down_ans.astype(np.int)

In [16]:
#입력 : inputdata1   (587, 10, 6))
#정답 : up_or_down_ans (587,)

In [17]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [18]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [19]:
#train : 500 test : 87
inputdata1_train = inputdata1[:550]
inputdata1_test = inputdata1[550:]

up_or_down_train = up_or_down_ans[:550]
up_or_down_test =up_or_down_ans[550:]

In [20]:
class TrainDataset(Dataset):
  def __init__(self):
    self.x_data = dummy_input
    self.y_data = up_or_down_ans_dummy

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [21]:
class TestDataset(Dataset):
  def __init__(self):
    self.x_data = inputdata1_test
    self.y_data = up_or_down_test

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.tensor(self.x_data[idx])
    y = torch.tensor(self.y_data[idx])
    return x, y

In [22]:
train_dataset = TrainDataset()
test_dataset = TestDataset()
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False)


In [23]:
# feature 갯수
input_size = 2

#output size 바로 결과 예측을 위해서 1로 설정 
hidden_dim = 2

batch_size = 1 

# 15일
sequence_length = 3

# one layer rnn이다.
n_layers = 1

#inputs = (batch_size * seq_len * input_size ) 인 행렬\
#hidden = (num_layers * batch_size * hidden_size ) 인 행렬
#output = (batch,seq_len,hidden)

In [24]:
#상승 또는 하락 판별 모델
class Model(nn.Module):
    def __init__(self, input_size, hidden_dim,output_size ,n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        h0 = self.init_hidden(batch_size)
        c0 = self.init_hidden(batch_size)
        

        # Passing in the input and hidden state into the model and obtaining outputs
        out, (hn,cn) = self.rnn(x, (h0,c0))
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        
        #out = self.fc(out)
        #out = out[:,-1,:].view(1,1,1)
        return out[:,-1,:],
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.rand(self.n_layers, batch_size, self.hidden_dim)
        hidden = nn.init.xavier_normal_(hidden).to(device)
        return hidden

In [74]:
#모델 초기화
model = Model(input_size, hidden_dim, output_size=1, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
nb_epochs = 1000
lr=0.005

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [75]:
#학습
#for epoch in range(nb_epochs + 1):
#     for batch_idx, samples in enumerate(dataloader):
for epoch in range(nb_epochs + 1):
    print("epoch: ", epoch)
    for batch_idx, (input_data,label) in enumerate(dataloader):
        optimizer.zero_grad()
        input_data = input_data.float().to(device)
        label = label.view(-1).long().to(device)
        output, = model(input_data)
        #print(input_data.shape)
        #print(output.shape)
        #print(label.shape)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        if(batch_idx%10==0):
            print("epoch: ",epoch, "  iter:",batch_idx,"/502")
            print("Loss: {:.4f}".format(loss.item()))
            print("")

epoch:  0
epoch:  0   iter: 0 /502
Loss: 0.5681

epoch:  1
epoch:  1   iter: 0 /502
Loss: 0.5545

epoch:  2
epoch:  2   iter: 0 /502
Loss: 0.5454

epoch:  3
epoch:  3   iter: 0 /502
Loss: 0.5610

epoch:  4
epoch:  4   iter: 0 /502
Loss: 0.5524

epoch:  5
epoch:  5   iter: 0 /502
Loss: 0.5641

epoch:  6
epoch:  6   iter: 0 /502
Loss: 0.5726

epoch:  7
epoch:  7   iter: 0 /502
Loss: 0.5585

epoch:  8
epoch:  8   iter: 0 /502
Loss: 0.5469

epoch:  9
epoch:  9   iter: 0 /502
Loss: 0.5648

epoch:  10
epoch:  10   iter: 0 /502
Loss: 0.5444

epoch:  11
epoch:  11   iter: 0 /502
Loss: 0.5570

epoch:  12
epoch:  12   iter: 0 /502
Loss: 0.5500

epoch:  13
epoch:  13   iter: 0 /502
Loss: 0.5725

epoch:  14
epoch:  14   iter: 0 /502
Loss: 0.5510

epoch:  15
epoch:  15   iter: 0 /502
Loss: 0.5519

epoch:  16
epoch:  16   iter: 0 /502
Loss: 0.5612

epoch:  17
epoch:  17   iter: 0 /502
Loss: 0.5515

epoch:  18
epoch:  18   iter: 0 /502
Loss: 0.5580

epoch:  19
epoch:  19   iter: 0 /502
Loss: 0.5500



epoch:  167   iter: 0 /502
Loss: 0.2292

epoch:  168
epoch:  168   iter: 0 /502
Loss: 0.2293

epoch:  169
epoch:  169   iter: 0 /502
Loss: 0.2294

epoch:  170
epoch:  170   iter: 0 /502
Loss: 0.2289

epoch:  171
epoch:  171   iter: 0 /502
Loss: 0.2280

epoch:  172
epoch:  172   iter: 0 /502
Loss: 0.2284

epoch:  173
epoch:  173   iter: 0 /502
Loss: 0.2272

epoch:  174
epoch:  174   iter: 0 /502
Loss: 0.2624

epoch:  175
epoch:  175   iter: 0 /502
Loss: 0.2266

epoch:  176
epoch:  176   iter: 0 /502
Loss: 0.2263

epoch:  177
epoch:  177   iter: 0 /502
Loss: 0.2259

epoch:  178
epoch:  178   iter: 0 /502
Loss: 0.2255

epoch:  179
epoch:  179   iter: 0 /502
Loss: 0.2255

epoch:  180
epoch:  180   iter: 0 /502
Loss: 0.2254

epoch:  181
epoch:  181   iter: 0 /502
Loss: 0.2247

epoch:  182
epoch:  182   iter: 0 /502
Loss: 0.2242

epoch:  183
epoch:  183   iter: 0 /502
Loss: 0.2239

epoch:  184
epoch:  184   iter: 0 /502
Loss: 0.2241

epoch:  185
epoch:  185   iter: 0 /502
Loss: 0.2241

epoch

epoch:  324
epoch:  324   iter: 0 /502
Loss: 0.2094

epoch:  325
epoch:  325   iter: 0 /502
Loss: 0.2094

epoch:  326
epoch:  326   iter: 0 /502
Loss: 0.2092

epoch:  327
epoch:  327   iter: 0 /502
Loss: 0.2091

epoch:  328
epoch:  328   iter: 0 /502
Loss: 0.2088

epoch:  329
epoch:  329   iter: 0 /502
Loss: 0.2091

epoch:  330
epoch:  330   iter: 0 /502
Loss: 0.2091

epoch:  331
epoch:  331   iter: 0 /502
Loss: 0.2069

epoch:  332
epoch:  332   iter: 0 /502
Loss: 0.2090

epoch:  333
epoch:  333   iter: 0 /502
Loss: 0.2081

epoch:  334
epoch:  334   iter: 0 /502
Loss: 0.2077

epoch:  335
epoch:  335   iter: 0 /502
Loss: 0.2088

epoch:  336
epoch:  336   iter: 0 /502
Loss: 0.2079

epoch:  337
epoch:  337   iter: 0 /502
Loss: 0.2342

epoch:  338
epoch:  338   iter: 0 /502
Loss: 0.2081

epoch:  339
epoch:  339   iter: 0 /502
Loss: 0.2336

epoch:  340
epoch:  340   iter: 0 /502
Loss: 0.2086

epoch:  341
epoch:  341   iter: 0 /502
Loss: 0.2085

epoch:  342
epoch:  342   iter: 0 /502
Loss: 0

epoch:  482
epoch:  482   iter: 0 /502
Loss: 0.2045

epoch:  483
epoch:  483   iter: 0 /502
Loss: 0.2047

epoch:  484
epoch:  484   iter: 0 /502
Loss: 0.2042

epoch:  485
epoch:  485   iter: 0 /502
Loss: 0.2025

epoch:  486
epoch:  486   iter: 0 /502
Loss: 0.2046

epoch:  487
epoch:  487   iter: 0 /502
Loss: 0.2045

epoch:  488
epoch:  488   iter: 0 /502
Loss: 0.2046

epoch:  489
epoch:  489   iter: 0 /502
Loss: 0.2046

epoch:  490
epoch:  490   iter: 0 /502
Loss: 0.2042

epoch:  491
epoch:  491   iter: 0 /502
Loss: 0.2187

epoch:  492
epoch:  492   iter: 0 /502
Loss: 0.2044

epoch:  493
epoch:  493   iter: 0 /502
Loss: 0.2044

epoch:  494
epoch:  494   iter: 0 /502
Loss: 0.2044

epoch:  495
epoch:  495   iter: 0 /502
Loss: 0.2039

epoch:  496
epoch:  496   iter: 0 /502
Loss: 0.2043

epoch:  497
epoch:  497   iter: 0 /502
Loss: 0.2038

epoch:  498
epoch:  498   iter: 0 /502
Loss: 0.2043

epoch:  499
epoch:  499   iter: 0 /502
Loss: 0.2022

epoch:  500
epoch:  500   iter: 0 /502
Loss: 0

epoch:  639   iter: 0 /502
Loss: 0.2020

epoch:  640
epoch:  640   iter: 0 /502
Loss: 0.2020

epoch:  641
epoch:  641   iter: 0 /502
Loss: 0.2020

epoch:  642
epoch:  642   iter: 0 /502
Loss: 0.2020

epoch:  643
epoch:  643   iter: 0 /502
Loss: 0.2020

epoch:  644
epoch:  644   iter: 0 /502
Loss: 0.2020

epoch:  645
epoch:  645   iter: 0 /502
Loss: 0.2019

epoch:  646
epoch:  646   iter: 0 /502
Loss: 0.2020

epoch:  647
epoch:  647   iter: 0 /502
Loss: 0.2144

epoch:  648
epoch:  648   iter: 0 /502
Loss: 0.2019

epoch:  649
epoch:  649   iter: 0 /502
Loss: 0.2126

epoch:  650
epoch:  650   iter: 0 /502
Loss: 0.2020

epoch:  651
epoch:  651   iter: 0 /502
Loss: 0.2019

epoch:  652
epoch:  652   iter: 0 /502
Loss: 0.2017

epoch:  653
epoch:  653   iter: 0 /502
Loss: 0.2019

epoch:  654
epoch:  654   iter: 0 /502
Loss: 0.2019

epoch:  655
epoch:  655   iter: 0 /502
Loss: 0.2016

epoch:  656
epoch:  656   iter: 0 /502
Loss: 0.2018

epoch:  657
epoch:  657   iter: 0 /502
Loss: 0.2018

epoch

epoch:  796
epoch:  796   iter: 0 /502
Loss: 0.2008

epoch:  797
epoch:  797   iter: 0 /502
Loss: 0.2008

epoch:  798
epoch:  798   iter: 0 /502
Loss: 0.1996

epoch:  799
epoch:  799   iter: 0 /502
Loss: 0.2007

epoch:  800
epoch:  800   iter: 0 /502
Loss: 0.2008

epoch:  801
epoch:  801   iter: 0 /502
Loss: 0.2004

epoch:  802
epoch:  802   iter: 0 /502
Loss: 0.2002

epoch:  803
epoch:  803   iter: 0 /502
Loss: 0.2037

epoch:  804
epoch:  804   iter: 0 /502
Loss: 0.2019

epoch:  805
epoch:  805   iter: 0 /502
Loss: 0.2007

epoch:  806
epoch:  806   iter: 0 /502
Loss: 0.2005

epoch:  807
epoch:  807   iter: 0 /502
Loss: 0.2004

epoch:  808
epoch:  808   iter: 0 /502
Loss: 0.2007

epoch:  809
epoch:  809   iter: 0 /502
Loss: 0.2047

epoch:  810
epoch:  810   iter: 0 /502
Loss: 0.2001

epoch:  811
epoch:  811   iter: 0 /502
Loss: 0.2006

epoch:  812
epoch:  812   iter: 0 /502
Loss: 0.2006

epoch:  813
epoch:  813   iter: 0 /502
Loss: 0.1997

epoch:  814
epoch:  814   iter: 0 /502
Loss: 0

epoch:  955
epoch:  955   iter: 0 /502
Loss: 0.1995

epoch:  956
epoch:  956   iter: 0 /502
Loss: 0.1995

epoch:  957
epoch:  957   iter: 0 /502
Loss: 0.1995

epoch:  958
epoch:  958   iter: 0 /502
Loss: 0.1994

epoch:  959
epoch:  959   iter: 0 /502
Loss: 0.1994

epoch:  960
epoch:  960   iter: 0 /502
Loss: 0.1994

epoch:  961
epoch:  961   iter: 0 /502
Loss: 0.1994

epoch:  962
epoch:  962   iter: 0 /502
Loss: 0.2034

epoch:  963
epoch:  963   iter: 0 /502
Loss: 0.1994

epoch:  964
epoch:  964   iter: 0 /502
Loss: 0.1994

epoch:  965
epoch:  965   iter: 0 /502
Loss: 0.2056

epoch:  966
epoch:  966   iter: 0 /502
Loss: 0.1993

epoch:  967
epoch:  967   iter: 0 /502
Loss: 0.1993

epoch:  968
epoch:  968   iter: 0 /502
Loss: 0.1994

epoch:  969
epoch:  969   iter: 0 /502
Loss: 0.1994

epoch:  970
epoch:  970   iter: 0 /502
Loss: 0.1998

epoch:  971
epoch:  971   iter: 0 /502
Loss: 0.1994

epoch:  972
epoch:  972   iter: 0 /502
Loss: 0.1995

epoch:  973
epoch:  973   iter: 0 /502
Loss: 0

In [ ]:
with torch.no_grad():
    count = 0
    for i in range(550):
        ip, ans = train_dataset.__getitem__(i)
        ip = ip.float().view(-1,10,feature_count).to(device)
        output, = model(ip)
        a,p = torch.max(output,1)
        if(p.item() == ans.item()):
            count += 1
            #print("right")
    acc = count/550.0
    print(acc)